In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
df = pd.read_csv("data/merged_movie_metadata.csv")
df.head(2)

In [ ]:
plot_summaries_path = 'data/plot_summaries.txt'
plot_summaries = {}
with open(plot_summaries_path, 'r', encoding='utf-8') as file:
    for line in file:
        parts = line.strip().split('\t', 1)
        if len(parts) == 2:
            wiki_id, summary = parts
            plot_summaries[int(wiki_id)] = summary
            

# add plot_summaries.txt
df['Plot_Summary_Base'] = df['Wikipedia_ID'].map(plot_summaries)

# take only unique Wikipedia_ID
df['Wikipedia_ID'].nunique()

df.head()

In [ ]:
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def tok(sentence):
    tokenized = word_tokenize(sentence)
    return [lemmatizer.lemmatize(word.lower()) for word in tokenized if word.isalpha()]

In [ ]:
tokenized_plots_america = df[df['Continents'].str.contains('America')].apply(
    lambda row: tok(row['Plot_Summary_Base']) if pd.notna(row['Plot_Summary_Base']) 
    else tok(row['Plot']) if pd.notna(row['Plot']) else np.nan,
    axis=1
)

In [ ]:
tokenized_plots_europe = df[df['Continents'].str.contains('Europe')].apply(
    lambda row: tok(row['Plot_Summary_Base']) if pd.notna(row['Plot_Summary_Base']) 
    else tok(row['Plot']) if pd.notna(row['Plot']) else np.nan,
    axis=1
)

In [ ]:
tokenized_plots_both = df[df['Continents'].str.contains('Both')].apply(
    lambda row: tok(row['Plot_Summary_Base']) if pd.notna(row['Plot_Summary_Base']) 
    else tok(row['Plot']) if pd.notna(row['Plot']) else np.nan,
    axis=1
)

In [ ]:
from collections import Counter

word_count_america = Counter(tokenized_plots_america.dropna().explode())
word_count_europe = Counter(tokenized_plots_europe.dropna().explode())
word_count_both = Counter(tokenized_plots_both.dropna().explode())

In [ ]:
print("Top 10 words in America:")
print(word_count_america.most_common(10))

print("\nTop 10 words in Europe:")
print(word_count_europe.most_common(10))

print("\nTop 10 words in Both:")
print(word_count_both.most_common(10))

In [ ]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def filter_stop_words(word_list):
    return [word for word in word_list if word not in stop_words]

filtered_word_count_america = Counter(filter_stop_words(tokenized_plots_america.dropna().explode()))
filtered_word_count_europe = Counter(filter_stop_words(tokenized_plots_europe.dropna().explode()))
filtered_word_count_both = Counter(filter_stop_words(tokenized_plots_both.dropna().explode()))

print("\nTop 10 interesting words in America:")
print(filtered_word_count_america.most_common(10))

print("\nTop 10 interesting words in Europe:")
print(filtered_word_count_europe.most_common(10))

print("\nTop 10 interesting words in Both:")
print(filtered_word_count_both.most_common(10))

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def generate_word_cloud(word_count, title):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_count)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(title)
    plt.show()


generate_word_cloud(filtered_word_count_america, "Word Cloud for America")
generate_word_cloud(filtered_word_count_europe, "Word Cloud for Europe")
generate_word_cloud(filtered_word_count_both, "Word Cloud for Both")

#### To Do:
- Filtrer encore parce que les mots ne sont pas hyper intéressants
- Faire le word cloud en forme des continents (j'ai essayé mais c'était affreux)
